https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7924484/

https://codingfleet.com/transformation-details/determining-gender-by-name-with-python/

In [1]:
import pandas as pd

In [2]:
import wosplus as wp

In [3]:
wos=wp.wosplus('drive.cfg') 

In [4]:
df=wos.read_drive_excel('produccionUdeA.xlsx',dtype={'COD_RH':str})

In [5]:
pd.set_option('display.max_columns', 500)

In [6]:
TPO_NAL={'C':'colombia', 'H':'chile', 'M':'mexico', 'B':'Brazil', 'E':'other_countries', 
 'V':'venezuela', 'Q':'ecuador', 'U':'cuba', 'R':'peru', 'P':'portugal'}

In [7]:
df['author_TPO_NACIONALIDAD'].unique()

array(['C', 'H', 'M', 'B', 'E', 'V', 'Q', 'U', 'R', 'P'], dtype=object)

In [8]:
df[df['author_TPO_NACIONALIDAD']=='P']['COD_RH'].unique()

array(['0001110012', '0001607026'], dtype=object)

In [9]:
df[['author_TXT_TOTAL_NAMES','author_TXT_NAMES_RH','author_TPO_NACIONALIDAD','author_TPO_SEXO']]

,author_TXT_TOTAL_NAMES,author_TXT_NAMES_RH,author_TPO_NACIONALIDAD,author_TPO_SEXO
0,NATALIA ANDREA ROJAS HENAO,NATALIA ANDREA,C,F
1,NATALIA ANDREA ROJAS HENAO,NATALIA ANDREA,C,F
2,NATALIA ANDREA ROJAS HENAO,NATALIA ANDREA,C,F
3,NATALIA ANDREA ROJAS HENAO,NATALIA ANDREA,C,F
4,Jefferson Alejandro Perez Mesa,Jefferson Alejandro,C,M
...,...,...,...,...
95482,Sindy Dayana Jojoa SIERRA,Sindy Dayana,C,F
95483,Jose David Ruiz Alvarez,Jose David,C,M
95484,Luis Alfredo Munoz Hernandez,Luis Alfredo,C,M
95485,Oscar Alexander Gutierrez Lesmes,Oscar Alexander,C,M


In [10]:
import gender_guesser.detector as GENDER

class Detector:
    '''
    * Works for all countries
    * convert mostly_(fe)male to (fe)male
    * Works for composed names by chosing the female or male of each one (in that order) 
      and ignoring the other possibilities
    '''
    case_sensitive = False
    D=GENDER.Detector(case_sensitive=case_sensitive)
    LA_countries=['Brazil', 'Mexico', 'Argentina','Chile', 'Colombia','Bolivia','Cuba', #5
              'Costa Rica', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', #10
               'Nicaragua', 'Panama', 'Paraguay', 'Peru', #15
              'Dominican Republic','Uruguay','Venezuela']

    def get_country(self,country,countries=D.__class__.COUNTRIES,la_countries=LA_countries):
        if country.lower() in [s.lower() for s  in la_countries if s.lower()!='brazil']:
            return 'spain'
        elif country.lower() == 'brazil':
            return 'portugal'
        elif country.lower() in countries:
            return country.lower()
        else:
            return None
    
    def get_gender(self,names,country=None,**kwargs):
        if country:
            country=self.get_country(country)
            g=[ self.D.get_gender(name, country=u'{}'.format(country)) for name in names.split() ]
        else:
            g=[ self.D.get_gender(name) for name in names.split()]
            g=[ x.replace('mostly_','') for x in g]
        male_female = list(set(['male','female']).intersection(g))
        if male_female:
            return male_female[0]
        else:
            return sorted(g)[0]
            
d=Detector()
assert d.get_gender('Andrea',country='italy') == 'male'
assert d.get_gender('Andrea',country='spain') == 'female'
assert d.get_gender('Andrea',country='colombia') == 'female'

In [146]:
d.get_gender('Diego Alejandro',country='italy')

'male'

In [151]:
d.get_gender('Diego Alejandro',country=)

'andy'

In [10]:
t={'female':'F', 'male':'M', 'unknown':'U', 'mostly_female':'F', 'mostly_male':'M',
       'andy':'A'}

In [11]:
def get_gender(L):
    if 'F' in L:
        return 'F'
    elif 'M' in L:
        return 'M'
    else:
        return 'U'

In [12]:
df=df.drop_duplicates('author_TXT_NAMES_RH').reset_index(drop=True)

In [13]:
df.shape

(3967, 74)

In [14]:
import gender_guesser.detector as GENDER
D=GENDER.Detector(case_sensitive=False)

In [57]:
LA_countries=['Brazil', 'Mexico', 'Argentina','Chile', 'Colombia','Bolivia','Cuba', #5
              'Costa Rica', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', #10
               'Nicaragua', 'Panama', 'Paraguay', 'Peru', #15
              'Dominican Republic','Uruguay','Venezuela']
def get_country(country,countries=D.__class__.COUNTRIES,la_countries=LA_countries):
        if country.lower() in [s.lower() for s  in la_countries if s.lower()!='brazil']:
            return 'spain'
        elif country.lower() == 'brazil':
            return 'portugal'
        elif country.lower() in countries:
            return country.lower()
        else:
            return None

In [91]:
get_country('colombia')

'spain'

In [100]:
def get_gender_country(name,country='spain'):
    country = get_country(country)
    fm=D.get_gender(name,country)
    fm=fm.replace('mostly_','')
    if fm not in ['male','female']:
        fm = D.get_gender(name)
    return fm

In [99]:
get_gender_country('diego',country='colombia')

spain


'male'

In [110]:
df['gg1'] = df.apply(lambda x: D.get_gender( x.get('author_TXT_NAMES_RH').split()[0] ) ,axis='columns') #TPO_NAL[ x.get('author_TPO_NACIONALIDAD') ]
df['gg2'] = df.apply(lambda x: D.get_gender( x.get('author_TXT_NAMES_RH').split()[-1]   ) ,axis='columns') #TPO_NAL[ x.get('author_TPO_NACIONALIDAD')]

In [120]:
df['gg1'] = df.apply(lambda x: get_gender_country( x.get('author_TXT_NAMES_RH').split()[0] ) ,axis='columns') #TPO_NAL[ x.get('author_TPO_NACIONALIDAD') ]
df['gg2'] = df.apply(lambda x: get_gender_country( x.get('author_TXT_NAMES_RH').split()[-1]   ) ,axis='columns') #TPO_NAL[ x.get('author_TPO_NACIONALIDAD')]

In [129]:
df['gg1'] = df.apply(lambda x: get_gender_country( x.get('author_TXT_NAMES_RH').split()[0],
                  country=TPO_NAL[ x.get('author_TPO_NACIONALIDAD') ] ) ,axis='columns') 
df['gg2'] = df.apply(lambda x: get_gender_country( x.get('author_TXT_NAMES_RH').split()[-1],
                  country=TPO_NAL[ x.get('author_TPO_NACIONALIDAD') ] ) ,axis='columns') 

In [130]:
df['gg1']=df['gg1'].apply(lambda d: t[d]) 
df['gg2']=df['gg2'].apply(lambda d: t[d]) 

In [131]:
df['gg']=df.apply(lambda d: get_gender( [d['gg1'],d['gg2']]) ,axis='columns')

In [132]:
df[['author_TXT_TOTAL_NAMES','author_TXT_NAMES_RH','author_TPO_NACIONALIDAD','author_TPO_SEXO','gg']#,'gg1_es','gg2_es','gg1_us','gg2_us']
  ].drop_duplicates().sample(50)#[:50]

,author_TXT_TOTAL_NAMES,author_TXT_NAMES_RH,author_TPO_NACIONALIDAD,author_TPO_SEXO,gg
2894,DIANA PATRICIA GIRALDO MENDEZ,DIANA PATRICIA,C,F,F
3391,PAOLA ROCIO FARFAN ARDILA,PAOLA ROCIO,C,F,F
3307,Leydi Camila Rodriguez Corredor,Leydi Camila,C,F,F
934,Oscar Arturo Zapata Norena,Oscar Arturo,C,M,M
3837,Cesar Anibal Olmos Severiche,Cesar Anibal,C,M,M
2435,Adry Liliana Manrique Lagos,Adry Liliana,C,F,F
2338,Carlos Jaime Velasquez Franco,Carlos Jaime,C,M,M
133,Gabriel Angel Quiroz Herrera,Gabriel Angel,C,M,M
158,Jhony Heriberto Giraldo Zuluaga,Jhony Heriberto,C,M,M
679,Yuliana Andrea Osorio Osorno,Yuliana Andrea,C,F,F


In [133]:
df.shape

(3967, 77)

In [134]:
T=df[df['author_TPO_SEXO']==df['gg']].shape[0]
T

3626

In [135]:
U=df[df['gg']=='U'].shape[0]
U

303

In [136]:
E=df[ (df['author_TPO_SEXO']!=df['gg']) & (df['gg']!='U') ][['author_TXT_TOTAL_NAMES','author_TXT_NAMES_RH',
                                                           'author_TPO_NACIONALIDAD','author_TPO_SEXO','gg']].shape[0]
E

38

In [137]:
E/T #0.13 to 0.1 with Spain

0.010479867622724766

In [119]:
U/T #→ 0.2

0.08379424778761062